<a href="https://colab.research.google.com/github/kuanyshbakytuly/dataton/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import pandas as pd
!pip install torchsummary

In [ ]:
#Collecting labels from "train_labels.csv" and saving them in upd_dictionary

df = pd.read_csv("/train_labels.csv")
df.rename(columns = {'2418.jpg': 'image'}, inplace = True)
df.rename(columns = {'1': 'Y'}, inplace = True)
dicts = df.to_dict(orient="index")

upd_dict = {}
for k, v in dicts.items():
  for k1, v1 in v.items():
    if len(k1) == 1:
      upd_dict[keys] = v1
    else:
      keys = v1

upd_dict["2418.jpg"] = 1

In [27]:
#folders
train_data = "/content/train_imgs"
valid_data = "/content/train_imgs"
test_data ="/content/test_imgs"

In [28]:
from torch.utils.data import DataLoader, Dataset
class Data(Dataset):
    #Providing the class that returns images and targets from folder

    def __init__(self, folder, lower_bound=0, upper_bound=-1):
        #Initializing from folder with lower_bound and upper_bound
        images_in_folder =  glob(folder + '/*.jpg')
        self.images = images_in_folder[lower_bound: upper_bound] 
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        self.targets = [upd_dict[image_path.split('/')[-1]] for image_path in self.images]

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, i):
        #Getting images[i] and its target
        image_path = self.images[i]
        target = self.targets[i]

        im = (cv2.imread(image_path)[:,:,::-1])
        im = self.image_segmentation(im)
        im = cv2.resize(im, (224,224))
        im = torch.tensor(im/255)
        im = im.permute(2,0,1)
        im = self.normalize(im) 

        return im.float().to(device), torch.tensor([target]).float().to(device)

    def image_segmentation(self, image):
        #Returning edited images with fire segmentation
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_hsv = np.array([0,0,250]) #orange
        upper_hsv = np.array([250,255,255]) #black
    
        mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        output = cv2.bitwise_and(image, image, mask = mask) / 255
        image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
        image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
        return image_sharp

In [29]:
def get_model():
    #Returning the pretrained model - 
    model = models.vgg19(pretrained=True)

    #Freezing all parameters
    for param in model.parameters():
        param.requires_grad = False

    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())

    #loss_function - Binary Cross Entropy
    #optimizer - Adam
    #Recommended by https://www.mdpi.com/1424-8220/22/5/1701 for Fire Classifying
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)
    return model.to(device), loss_fn, optimizer
    return model.to(device), loss_fn, optimizer

In [31]:
def get_data():
    #Returning training and validation DataLoader 
    lower = 0
    upper = 2500
    train = Data(train_data, lower, upper)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True, drop_last = True)
    val = Data(valid_data, upper, upper+200)
    val_dl = DataLoader(val, batch_size=32, shuffle=True, drop_last = True)
    test = Data(test_data)
    tst_dl = DataLoader(val, batch_size=len(test), shuffle=True, drop_last = True)
    return trn_dl, val_dl, tst_dl

In [32]:
def train_batch(x, y, model, opt, loss_fn):
    #Returns loss variables, optimizing after each step
    optimizer.zero_grad()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    #Evaluating the model without training it
    prediction = model(x)

    #if 0<prediction<0.5 => return False
    #else True
    is_correct = (prediction > 0.5) == y
    return is_correct.cpu().numpy().tolist()

In [ ]:
trn_dl, val_dl, tst_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
train_losses, train_accuracies = [], []
val_accuracies = []
for epoch in range(10):
    #Training the model over increasing epochs
    print(f" epoch {epoch + 1}/10")
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies = []

    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss) 
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        val_epoch_accuracies.extend(val_is_correct)
    val_epoch_accuracy = np.mean(val_epoch_accuracies)

    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)


In [ ]:
epochs = np.arange(2)+1
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline
plt.plot(epochs, train_accuracies, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracies, 'r', label='Validation accuracy')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.title('Training and validation accuracy with VGG16 \nand 2500 training data points')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0.9,1)
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
plt.legend()
plt.grid('off')
plt.show()

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
model, _ = get_model()
model.load_state_dict(torch.load('model.pth'))
model.eval()

In [ ]:
import csv
dict_test = {}
for ix, batch in enumerate(iter(tst_dl)):
    x, y = batch
    test_is_correct = accuracy(x, y, model)
    dict_test[y] = test_is_correct[0][0]

In [ ]:
with open('mycsvfile.csv', 'w') as file: 
    w = csv.DictWriter(file, dict_test.keys())
    w.writeheader()
    w.writerow(dict_test)